In [ ]:
import torch
from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.5.0+cu101 _CudaDeviceProperties(name='Tesla K80', major=3, minor=7, total_memory=11441MB, multi_processor_count=13)


Install Requirements

In [ ]:
%%shell

pip install cython

# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354

pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-mtvnb4q2
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-mtvnb4q2
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=266989 sha256=3b1c27ab3894bb27e5b63540ed567f11d6b1e1e9b5a0a8a9ff330f6cf165312f
  Stored in directory: /tmp/pip-ephem-wheel-cache-x3l1u0uu/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0.0
    Uninstalling pycocotools-2.0.0:
      Successfully uninstalled pycocotools-2.0.0


Download Pascal VOC 2012 Dataset


In [ ]:
# Download pascal voc 2012 dataset
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
!tar -xvf VOCtrainval_11-May-2012.tar
!rm VOCtrainval_11-May-2012.tar

Streaming output truncated to the last 5000 lines.
VOCdevkit/VOC2012/SegmentationClass/2008_001876.png
VOCdevkit/VOC2012/SegmentationClass/2008_001882.png
VOCdevkit/VOC2012/SegmentationClass/2008_001885.png
VOCdevkit/VOC2012/SegmentationClass/2008_001895.png
VOCdevkit/VOC2012/SegmentationClass/2008_001896.png
VOCdevkit/VOC2012/SegmentationClass/2008_001926.png
VOCdevkit/VOC2012/SegmentationClass/2008_001966.png
VOCdevkit/VOC2012/SegmentationClass/2008_001971.png
VOCdevkit/VOC2012/SegmentationClass/2008_001992.png
VOCdevkit/VOC2012/SegmentationClass/2008_001997.png
VOCdevkit/VOC2012/SegmentationClass/2008_002032.png
VOCdevkit/VOC2012/SegmentationClass/2008_002043.png
VOCdevkit/VOC2012/SegmentationClass/2008_002064.png
VOCdevkit/VOC2012/SegmentationClass/2008_002066.png
VOCdevkit/VOC2012/SegmentationClass/2008_002067.png
VOCdevkit/VOC2012/SegmentationClass/2008_002073.png
VOCdevkit/VOC2012/SegmentationClass/2008_002079.png
VOCdevkit/VOC2012/SegmentationClass/2008_002080.png
VOCdevkit/VOC

Convert Dataset format from Pascal VOC to COCO (Suit Input for SSD Implementations)

In [ ]:
!git clone https://github.com/ganeshkulkarni98/PascalVOC2COCO
%cd PascalVOC2COCO
!python voc2coco.py --annotation_folder_path /content/VOCdevkit/VOC2012/Annotations --json_file_path /content/VOCdevkit/VOC2012/coco_dataset.json
%cd ../

Cloning into 'PascalVOC2COCO'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 63 (delta 26), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (63/63), done.
/content/PascalVOC2COCO
Number of xml files: 17125
Success: /content/VOCdevkit/VOC2012/coco_dataset.json
/content


Take set of dataset for Training, Validation and Testing of SSD Model

In [ ]:
import json
import os
import shutil
# split factor for validation and testing is same
# if split_factor is 0.9 then 90 % train, 5% validation, 5% test

def create_dataset(coco_json_path, image_folder_path, split_factor, percentage_dataset):

  data = json.load(open(coco_json_path))
  os.mkdir('/content/data')                                          # data folder is created

  #percentage of dataset that you have to consider for train, val and test
  length_dataset = int(len(data['annotations']) * percentage_dataset / 100) 
  end_train = int(length_dataset * split_factor)

  for process in ['train','val','test']:

    if process == 'train':
      start = 0
      end = end_train - 1
    
    elif process == 'val' : 
      start = end_train
      end = start + int((100-split_factor*100)/200) * length_dataset - 1
    
    elif process == 'test' :
      start = start + int((100-split_factor*100)/200) * length_dataset
      end = length_dataset - 1

    image_id = []
    for i in data['annotations'][start:end]:
      image_id.append(i['image_id'])
    imgs = []
    images_details = []
    [imgs.append(x) for x in image_id if x not in imgs]
    for i in imgs:
      for j in data['images']:
        if j['id'] == i:
          imgs[imgs.index(i)] = data['images'][data['images'].index(j)]['file_name']
          images_details.append(j)
          break

    outputs = {}

    outputs['images'] = images_details
    outputs['annotations'] = data['annotations'][start:end]
    outputs['categories'] = data['categories']

    images_folder = '/content/data/' + process + '_images'
    os.mkdir(images_folder)

    for i in outputs['images']: 
      shutil.copyfile(os.path.join(image_folder_path, i['file_name']), os.path.join(images_folder, i['file_name']))

    # write json file
    json_file = '/content/data/' + process + '_coco_dataset.json'
    json.dump(outputs, open(json_file, 'w'))

In [ ]:
# split converted pascal voc dataset to coco into train, validation and testing
coco_json_path = '/content/VOCdevkit/VOC2012/coco_dataset.json'
image_folder_path = '/content/VOCdevkit/VOC2012/JPEGImages'
split_factor = 0.7
percentage_dataset = 100                   # conside 100% dataset for split into train, val and test

create_dataset(coco_json_path, image_folder_path, split_factor, percentage_dataset)

In [ ]:
#!rm -rf /content/data

Clone repository of SSD Implementation

In [ ]:
!git clone https://github.com/ganeshkulkarni98/ssd

Cloning into 'ssd'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 419 (delta 29), reused 0 (delta 0), pack-reused 363
Receiving objects: 100% (419/419), 176.40 MiB | 30.04 MiB/s, done.
Resolving deltas: 100% (213/213), done.


In [ ]:
%cd ssd

/content/ssd


SSD300 model pretrained weight (Trained on Pascal VOC dataset) : https://drive.google.com/file/d/14erG4YgA1EepHYjWiE3GGGbPT_y_9bb6/view?usp=sharing

In [ ]:
!cp /content/drive/My\ Drive/SSD300_pretrained_weight.pth /content/ssd/SSD300_pretrained_weight.pth

VGG16 (Backbone) pretrained weight (Trained on ImageNet Dataset) : https://download.pytorch.org/models/vgg16-397923af.pth

In [ ]:
!wget https://download.pytorch.org/models/vgg16-397923af.pth 

--2020-06-19 18:11:33--  https://download.pytorch.org/models/vgg16-397923af.pth
Resolving download.pytorch.org (download.pytorch.org)... 13.224.226.112, 13.224.226.28, 13.224.226.4, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.224.226.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 553433881 (528M) [binary/octet-stream]
Saving to: ‘vgg16-397923af.pth’

vgg16-397923af.pth  100%[===================>] 527.79M  20.0MB/s    in 27s     

2020-06-19 18:12:00 (19.4 MB/s) - ‘vgg16-397923af.pth’ saved [553433881/553433881]



Run Train Function 

In [ ]:
!python train.py

Loading model
Loading trained weight file...
model initialized
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
Epoch: [0][0/192]	Batch Time 26.676 (26.676)	Data Time 7.559 (7.559)	Training Loss 2.0225 Avg Training Loss (2.0225)	
Epoch: [0][8/192]	Batch Time 2.213 (5.136)	Data Time 0.001 (0.841)	Training Loss 2.3240 Avg Training Loss (2.1781)	
Epoch: [0][16/192]	Batch Time 2.210 (3.765)	Data Time 0.001 (0.445)	Training Loss 2.2843 Avg Training Loss (2.2348)	
Epoch: [0][24/192]	Batch Time 2.228 (3.265)	Data Time 0.001 (0.303)	Training Loss 1.6270 Avg Training Loss (2.1826)	
Epoch: [0][32/192]	Batch Time 2.143 (3.005)	Data Time 0.001 (0.230)	Training Loss 2.1814 Avg Training Loss (2.1596)	
Epoch: [0][40/192]	Batch Time 2.216 (2.847)	Data Time 0.001 (0.185)	Training Loss 1.9690 Avg Training Loss (2.1425)	
Epoch: [0][48/192]	Batch Time 2.2

Run Test function

In [ ]:
!python eval.py

Loading model
Loading trained weight file...
model initialized
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
creating index...
index created!
Evaluating:  99% 156/157 [01:18<00:00,  2.16it/s][8/157]	Validation Loss 7.316 Avg Val Loss(8.644)	
Evaluating: 100% 157/157 [01:19<00:00,  1.99it/s]
Total Average Validation Loss (8.644)	
{'aeroplane': 0.33044907450675964,
 'bicycle': 0.09734189510345459,
 'bird': 0.14675816893577576,
 'boat': 0.07972163707017899,
 'bottle': 0.09090909361839294,
 'bus': 0.3122660517692566,
 'car': 0.0886363685131073,
 'cat': 0.4871360659599304,
 'chair': 0.016913319006562233,
 'cow': 0.17373737692832947,
 'diningtable': 0.04545454680919647,
 'dog': 0.2978678345680237,
 'horse': 0.13589978218078613,
 'motorbike': 0.3494285047054291,
 'person': 0.11297774314880371,
 'pottedplant': 0.05681818351149559,
 'sheep':

Run Detection function

In [ ]:
!python detect.py

Streaming output truncated to the last 5000 lines.
       [[210, 245,  60],
        [210, 245,  60],
        [210, 245,  60],
        ...,
        [ 45,  45,  43],
        [ 47,  52,  46],
        [ 45,  52,  45]],

       [[ 91,  64,  53],
        [210, 245,  60],
        [210, 245,  60],
        ...,
        [ 52,  48,  47],
        [ 52,  48,  47],
        [ 54,  49,  46]]], dtype=uint8), array([[[48, 45, 56],
        [46, 44, 55],
        [44, 44, 56],
        ...,
        [12, 12, 12],
        [11, 11, 11],
        [11, 11, 11]],

       [[48, 48, 56],
        [48, 45, 56],
        [49, 43, 55],
        ...,
        [29, 29, 29],
        [28, 28, 28],
        [28, 28, 28]],

       [[48, 48, 56],
        [47, 46, 54],
        [47, 44, 55],
        ...,
        [28, 28, 28],
        [28, 28, 28],
        [28, 28, 28]],

       ...,

       [[16, 16, 14],
        [17, 17, 15],
        [14, 16, 15],
        ...,
        [ 3,  3,  3],
        [ 3,  3,  3],
        [ 4,  4,  4]],

    